In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import eviltransform
import datetime

# study area bounding box (bbox)
YMIN, XMIN = eviltransform.wgs2gcj(34.234, 108.9415)
YMAX, XMAX = eviltransform.wgs2gcj(34.241, 108.943)

# latitude bounds for segments immediately above and below bounding box (between bbox and intersection)
ABOVE = 34.241
BELOW = 34.231

# tighter latitude bounds for study road
LLEFT = 108.9467 
LRGHT = 108.9471

# bounds for west feeder road
WLEFT = 108.9393
WNRTH = 34.2355
WSOUT = 34.2345

# bound for east feeder road
ERGHT = 108.9486
ENRTH = 34.236
ESOUT = 34.237

# corrections for bbox speed estimation
NB_CORRECTION = 1.3
SB_CORRECTION = -2.6

# outlier threshold based on time needed/distance over study area and a sane speedlimit
D_MAX = 3000 # meters
T_MAX = pd.Timedelta(datetime.timedelta(minutes=10))
V_MAX = 150 #km/h

# grouper
GPR = pd.Grouper(freq="5T", key="timestamp1", closed="left", label="left", base=0)

# Identify data to process

Given the October and November range of our datasets, 

We're asked to predict peak hour traffic December 1st, a Thursday, so we want to train our algorithm on comparable periods in comparable days. This leads us to exclude weekends and Fridays from our data. We also note that the first week of October is a week of national holidays in China, thus we exclude that week from our dataset as well. There appear to be no other holidays in this time-period so that leaves us with
* 7 Thursdays
* 8 Mondays, Wednesdays, and Thursdays each

So these are the dates we wish to process.

# Processing methodology

1. 
1. Copy the dataset and remove the study area from the copy.

In [2]:
oct_mwf = ['10', '11', '12', '17', '18', '19', '24', '25', '26']
oct_thu = ['13', '20', '27']
nov_mwf = ['01', '02', '07', '08', '09', '14', '15', '16', '21', '22', '23', '28', '29', '30']
nov_thu = ['03', '10', '17', '24']

In [3]:
filename = 'data/xian/gps_201610{}'.format(oct_thu[0])

In [4]:
%%time

def read_gps(filename, has_header=False):
    # read and sort csv into order_id-then-chronological order
    df = pd.read_csv(filename, names=None if has_header else ['driver_id', 'order_id', 'timestamp', 'longitude', 'latitude'])
    df.drop(columns=['driver_id'], inplace=True)
    df.sort_values(['order_id', 'timestamp'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df

# dfy = read_gps(filename)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.63 µs


In [5]:
# dfy.head()

In [6]:
# get col name for lonlat pair
def which_lon_lat(which=1):
    lon = 'longitude{}'.format(which)
    lat = 'latitude{}'.format(which)
    return (lon, lat)   

# study area (bounding box) filter to find true values y
def bb_filter(df, which=1):
    lon, lat = which_lon_lat(which)
    return (df[lon] >= XMIN) & \
           (df[lon] <= XMAX) & \
           (df[lat] >= YMIN) & \
           (df[lat] <= YMAX)

# simulate given data by deleting all gps coords inside bbox
def delete_bbox(df):
    return df.loc[
        ~bb_filter(df, which='')
    ,:].copy()

# df = delete_bbox(dfy)
# len(df), len(dfy)

In [7]:
# am period
def am_filter(df, timestamp_col, after=False): # include 5 min before study time, include 5 minutes after period if requested
    return (df[timestamp_col] >= df[timestamp_col][0].replace(hour=5,minute=55,second=0)) & \
           ((df[timestamp_col] < df[timestamp_col][0].replace(hour=11,minute=5,second=0)) if (after) \
            else (df[timestamp_col] < df[timestamp_col][0].replace(hour=11,minute=0,second=0)))

# pm period
def pm_filter(df, timestamp_col, after=False): # include 5 min before
    return (df[timestamp_col] >= df[timestamp_col][0].replace(hour=15,minute=55,second=0)) & \
           ((df[timestamp_col] < df[timestamp_col][0].replace(hour=21,minute=5,second=0)) if (after) \
            else (df[timestamp_col] < df[timestamp_col][0].replace(hour=21,minute=0,second=0)))

In [8]:
%%time
def make_deltas_split_ampm(df, drop_outliers=True, after=False):
    # convert into gps steps
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Shanghai')
    df = pd.concat([df.rename(lambda x: x+'1', axis = 1),df.shift(-1).rename(lambda x: x+'2', axis = 1)],axis=1)
    df.drop(df[df['order_id1'] != df['order_id2']].index, inplace=True)
    df.drop(['order_id1', 'order_id2'], axis=1, inplace=True) # don't need these anymore
    if drop_outliers:
        df.drop(df.index[df['timestamp2'] - df['timestamp1'] > T_MAX], inplace=True)
    dfam = df.loc[am_filter(df, 'timestamp1', after),:].copy()
    dfpm = df.loc[pm_filter(df, 'timestamp1', after),:].copy()
#     dfam.sort_values("timestamp1",inplace=True)
#     dfam.reset_index(drop=True, inplace=True)
#     dfpm.sort_values("timestamp1",inplace=True)
#     dfpm.reset_index(drop=True, inplace=True)
    return dfam, dfpm

# dfam, dfpm = make_deltas_split_ampm(df, drop_outliers=True)
# del df
# dfyam, dfypm = make_deltas_split_ampm(dfy, drop_outliers=False, after=True)
# del dfy

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.39 µs


In [9]:
# dfpm.head()

In [10]:
def filter_y(df):
    # prefilter so we only calculate y speeds we need
    df.drop(df.index[
          ~(bb_filter(df, which=1) &
          bb_filter(df, which=2))
    ],inplace=True)
    return df

# dfyam = filter_y(dfyam)
# dfypm = filter_y(dfypm)
# dfyam.head()

In [11]:
%%time

def calc_speed(df, drop_outliers=True):
    # calculate distance and speed based on provided formula
    EARTH_RADIUS = 6371*1000  # m
    lat1 = np.radians(df['latitude1'])
    lat2 = np.radians(df['latitude2'])
    lon1 = np.radians(df['longitude1'])
    lon2 = np.radians(df['longitude2'])
    dlat = np.radians(df['latitude2']-df['latitude1'])
    dlon = np.radians(df['longitude2']-df['longitude1'])
    a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2) * np.sin(dlon/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    df['distance'] = EARTH_RADIUS * c
    if drop_outliers:
        df.drop(df.index[df['distance'] > D_MAX], inplace=True)
    df['speed'] = df['distance'] / (df['timestamp2'] - df['timestamp1']).dt.total_seconds() * 3.6
    if drop_outliers:
        df.drop(df.index[df['speed'] > V_MAX], inplace=True)
    return df

# dfam = calc_speed(dfam, drop_outliers=True)
# dfpm = calc_speed(dfpm, drop_outliers=True)
# dfyam = calc_speed(dfyam, drop_outliers=False)
# dfypm = calc_speed(dfypm, drop_outliers=False)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 7.39 µs


In [12]:
# dfam.head()

In [13]:
# filters for northbound or southbound (nb includes zeros as noted by fit in 1_exploration)
def y_dir_filter(df):
    return (df['latitude2']-df['latitude1']>=0)

# filters for northbound (excludes no movement)
def nbfilter(df):
    return (df['latitude2']-df['latitude1']>0)

# filters for southbound (excludes no movement)
def sbfilter(df):
    return (df['latitude2']-df['latitude1']<0)

# filters for no NS movement
def nnsfilter(df):
    return (df['latitude2']-df['latitude1']==0)

# filters for eastbound (excludes no movement)
def ebfilter(df):
    return (df['longitude2']-df['longitude1']>0)

# filters for westbound (excludes no movement)
def wbfilter(df):
    return (df['longitude2']-df['longitude1']<0)

# filters for no EW movement
def nwefilter(df):
    return (df['latitude2']-df['latitude1']==0)

In [14]:
# index placeholder to ensure full indices
def index_placeholder(timecol, am=True, after=False):
    start = timecol[0].replace(hour=5,minute=55,second=0) if am else timecol[0].replace(hour=15,minute=55,second=0)
    return pd.Series(index=[
        (start + i*datetime.timedelta(minutes=5)) for i in range(
            5*12+(2 if after else 1)
        )])

In [15]:
def make_y(dfam, dfpm):
    
    am_placeholder = index_placeholder(dfam['timestamp1'], am=True, after=True)
    pm_placeholder = index_placeholder(dfpm['timestamp1'], am=False, after=True)
    
    amidx = y_dir_filter(dfam)
    pmidx = y_dir_filter(dfpm)
    dfnam = dfam.loc[amidx,:]
    dfsam = dfam.loc[~amidx,:]
    dfnpm = dfpm.loc[pmidx,:]
    dfspm = dfpm.loc[~pmidx,:]
    
    dfnam = dfnam.groupby(GPR).mean()['speed'] + NB_CORRECTION
    dfsam = dfsam.groupby(GPR).mean()['speed'] + SB_CORRECTION
    dfnpm = dfnpm.groupby(GPR).mean()['speed'] + NB_CORRECTION
    dfspm = dfspm.groupby(GPR).mean()['speed'] + SB_CORRECTION
    
    dfam = pd.DataFrame({'p': am_placeholder, 'nb_speed': dfnam, 'sb_speed': dfsam})
    dfam.drop('p',axis=1,inplace=True)
    dfam.fillna(dfam.max(), inplace=True)
    dfpm = pd.DataFrame({'p': pm_placeholder, 'nb_speed': dfnpm, 'sb_speed': dfspm})
    dfpm.drop('p',axis=1,inplace=True)
    dfpm.fillna(dfpm.max(), inplace=True)
    
    return dfam, dfpm

# dfyam, dfypm = make_y(dfyam, dfypm)
# dfyam.head()

In [16]:
# dfypm.tail()

In [17]:
# north section above bbox
def n(df, which=1): # first lat/long in delta or second?
    lon, lat = which_lon_lat(which)
    return (df[lon] > XMIN) & \
           (df[lon] < XMAX) & \
           (df[lat] > YMAX) & \
           (df[lat] < ABOVE)

# south section below bbox
def s(df, which=1):
    lon, lat = which_lon_lat(which)
    return (df[lon] > XMIN) & \
           (df[lon] < XMAX) & \
           (df[lat] < YMIN) & \
           (df[lat] > BELOW)

# north intersection and up
def nn(df, which=1):
    lon, lat = which_lon_lat(which)
    return (df[lon] >= LLEFT) & \
           (df[lon] <= LRGHT) & \
           (df[lat] > ABOVE)

# south intersection and below
def ss(df, which=1):
    lon, lat = which_lon_lat(which)
    return (df[lon] >= LLEFT) & \
           (df[lon] <= LRGHT) & \
           (df[lat] < BELOW)

# western intersection 1 (northern one) left of bbox filter
def w(df, which=1):
    lon, lat = which_lon_lat(which)
    return (df[lon] < XMIN) & \
           (df[lon] > WLEFT) & \
           (df[lat] < WNRTH) & \
           (df[lat] > WSOUT)


# western intersection 2 (southern one) left of bbox filter
def e(df, which=1):
    lon, lat = which_lon_lat(which)
    return (df[lon] > XMAX) & \
           (df[lon] < ERGHT) & \
           (df[lat] > ENRTH) & \
           (df[lat] < ESOUT)

In [18]:
def make_x(dfam, dfpm):
    
    am_placeholder = index_placeholder(dfam['timestamp1'], am=True, after=False)
    pm_placeholder = index_placeholder(dfpm['timestamp1'], am=False, after=False)
    
    for i,df in enumerate([dfam, dfpm]):
        
        ### NORTHBOUND LINKS
        
        # n_n_nb: N -> N northbound
        nonzeros = df.loc[n(df,1) & n(df,2) & nbfilter(df),:].groupby(GPR)
        nonzeros_count = nonzeros.count()['speed']
        n_n_zeros_count = df.loc[n(df,1) & n(df,2) & nnsfilter(df),:].groupby(GPR).count()['speed']/2 # apply half of zeros to NB, other half to SB
        n_n_nb = nonzeros.mean()['speed']*nonzeros_count/(nonzeros_count + n_n_zeros_count)
        
        # n_nn: N -> NN
        n_nn = df.loc[n(df,1) & nn(df,2),:].groupby(GPR).mean()['speed']
        
        # s_n: S -> N
        s_n = df.loc[s(df,1) & n(df,2),:].groupby(GPR).mean()['speed']
        
        # s_s_nb: S -> S northbound
        nonzeros = df.loc[s(df,1) & s(df,2) & nbfilter(df),:].groupby(GPR)
        nonzeros_count = nonzeros.count()['speed']
        s_s_zeros_count = df.loc[s(df,1) & s(df,2) & nnsfilter(df),:].groupby(GPR).count()['speed']/2
        s_s_nb = nonzeros.mean()['speed']*nonzeros_count/(nonzeros_count + s_s_zeros_count)
        
        # ss_s: SS -> S
        ss_s = df.loc[ss(df,1) & s(df,2),:].groupby(GPR).mean()['speed']
        
        
        ### SOUTHBOUND LINKS
        
        # n_n_sb: N -> N southbound
        nonzeros = df.loc[n(df,1) & n(df,2) & sbfilter(df),:].groupby(GPR)
        nonzeros_count = nonzeros.count()['speed']
        n_n_sb = nonzeros.mean()['speed']*nonzeros_count/(nonzeros_count + n_n_zeros_count)
        
        # nn_n: NN -> N
        nn_n = df.loc[nn(df,1) & n(df,2),:].groupby(GPR).mean()['speed']
        
        # n_s: N -> S
        n_s = df.loc[n(df,1) & s(df,2),:].groupby(GPR).mean()['speed']
        
        # s_s_sb: S -> S southbound
        nonzeros = df.loc[s(df,1) & s(df,2) & sbfilter(df),:].groupby(GPR)
        nonzeros_count = nonzeros.count()['speed']
        s_s_sb = nonzeros.mean()['speed']*nonzeros_count/(nonzeros_count + s_s_zeros_count)
        
        # s_ss: S -> SS
        s_ss = df.loc[s(df,1) & ss(df,2),:].groupby(GPR).mean()['speed']
        
        
        ### MID LINKS
        
        # w_w_eb: W -> W eastbound
        nonzeros = df.loc[w(df,1) & w(df,2) & ebfilter(df),:].groupby(GPR)
        nonzeros_count = nonzeros.count()['speed']
        zeros_count = df.loc[w(df,1) & w(df,2) & nwefilter(df),:].groupby(GPR).count()['speed']/2
        w_w_eb = nonzeros.mean()['speed']*nonzeros_count/(nonzeros_count + zeros_count)
        
        # w_w_wb: W -> W westbound
        nonzeros = df.loc[w(df,1) & w(df,2) & wbfilter(df),:].groupby(GPR)
        nonzeros_count = nonzeros.count()['speed']
        w_w_wb = nonzeros.mean()['speed']*nonzeros_count/(nonzeros_count + zeros_count)
        
        # e_e_eb: E -> E eastbound
        nonzeros = df.loc[e(df,1) & e(df,2) & ebfilter(df),:].groupby(GPR)
        nonzeros_count = nonzeros.count()['speed']
        zeros_count = df.loc[e(df,1) & e(df,2) & nwefilter(df),:].groupby(GPR).count()['speed']/2
        e_e_eb = nonzeros.mean()['speed']*nonzeros_count/(nonzeros_count + zeros_count)
        
        # e_e_wb: E -> E westbound
        nonzeros = df.loc[e(df,1) & e(df,2) & wbfilter(df),:].groupby(GPR)
        nonzeros_count = nonzeros.count()['speed']
        e_e_wb = nonzeros.mean()['speed']*nonzeros_count/(nonzeros_count + zeros_count)
        
        
        ### log
        
        if i==0: # AM
            dfnam = pd.DataFrame({
                'p': am_placeholder,
                'n_n_nb': n_n_nb,
                'n_nn': n_nn,
                's_n': s_n,
                's_s_nb': s_s_nb,
                'ss_s': ss_s
            })
            dfnam.drop('p',axis=1,inplace=True)
            dfnam.fillna(dfnam.max(), inplace=True)
            dfsam = pd.DataFrame({
                'p': am_placeholder,
                'n_n_sb': n_n_sb,
                'nn_n': nn_n,
                'n_s': n_s,
                's_s_sb': s_s_sb,
                's_ss': s_ss
            })
            dfsam.drop('p',axis=1,inplace=True)
            dfsam.fillna(dfsam.max(), inplace=True)
            dfewam = pd.DataFrame({
                'p': am_placeholder,
                'w_w_eb': w_w_eb,
                'w_w_wb': w_w_wb,
                'e_e_eb': e_e_eb,
                'e_e_wb': e_e_wb
            })
            dfewam.drop('p',axis=1,inplace=True)
            dfewam.fillna(dfewam.max(), inplace=True)
        else: # PM
            dfnpm = pd.DataFrame({
                'p': pm_placeholder,
                'n_n_nb': n_n_nb,
                'n_nn': n_nn,
                's_n': s_n,
                's_s_nb': s_s_nb,
                'ss_s': ss_s
            })
            dfnpm.drop('p',axis=1,inplace=True)
            dfnpm.fillna(dfnpm.max(), inplace=True)
            dfspm = pd.DataFrame({
                'p': pm_placeholder,
                'n_n_sb': n_n_sb,
                'nn_n': nn_n,
                'n_s': n_s,
                's_s_sb': s_s_sb,
                's_ss': s_ss
            })
            dfspm.drop('p',axis=1,inplace=True)
            dfspm.fillna(dfspm.max(), inplace=True)
            dfewpm = pd.DataFrame({
                'p': pm_placeholder,
                'w_w_eb': w_w_eb,
                'w_w_wb': w_w_wb,
                'e_e_eb': e_e_eb,
                'e_e_wb': e_e_wb
            })
            dfewpm.drop('p',axis=1,inplace=True)
            dfewpm.fillna(dfewpm.max(), inplace=True)
            
    return dfnam, dfsam, dfewam, dfnpm, dfspm, dfewpm

# dfnam, dfsam, dfewam, dfnpm, dfspm, dfewpm = make_x(dfam, dfpm)
# dfewpm.head()

In [19]:
# del dfam, dfpm, dfnam, dfsam, dfewam, dfnpm, dfspm, dfewpm

In [20]:
%%time
def process_date(date, make_y_df=True, has_header=False):
    dfy = read_gps('data/xian/gps_2016{}'.format(date[0]+date[1]), has_header)
    df = delete_bbox(dfy)
    if make_y_df:
        dfyam, dfypm = make_deltas_split_ampm(dfy, drop_outliers=False, after=True)
        del dfy
        dfyam = filter_y(dfyam)
        dfypm = filter_y(dfypm)
        dfyam = calc_speed(dfyam, drop_outliers=False)
        dfypm = calc_speed(dfypm, drop_outliers=False)
        dfyam, dfypm = make_y(dfyam, dfypm)
        dfyam.to_pickle('data/processed/{}-{}_y_am.pkl'.format(date[0],date[1]))
        del dfyam
        dfypm.to_pickle('data/processed/{}-{}_y_pm.pkl'.format(date[0],date[1]))
        del dfypm
    else:
        del dfy
    dfam, dfpm = make_deltas_split_ampm(df, drop_outliers=True)
    del df
    dfam = calc_speed(dfam, drop_outliers=True)
    dfpm = calc_speed(dfpm, drop_outliers=True)
    dfnam, dfsam, dfewam, dfnpm, dfspm, dfewpm = make_x(dfam, dfpm)
    dfnam.to_pickle('data/processed/{}-{}_n_am.pkl'.format(date[0],date[1]))
    del dfnam
    dfsam.to_pickle('data/processed/{}-{}_s_am.pkl'.format(date[0],date[1]))
    del dfsam
    dfewam.to_pickle('data/processed/{}-{}_ew_am.pkl'.format(date[0],date[1]))
    del dfewam
    dfnpm.to_pickle('data/processed/{}-{}_n_pm.pkl'.format(date[0],date[1]))
    del dfnpm
    dfspm.to_pickle('data/processed/{}-{}_s_pm.pkl'.format(date[0],date[1]))
    del dfspm
    dfewpm.to_pickle('data/processed/{}-{}_ew_pm.pkl'.format(date[0],date[1]))
    del dfewpm

# process_date(('10','10'))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.11 µs


In [21]:
# %%time
# for filename in [('10',date) for date in oct_mwf+oct_thu] + [('11',date) for date in nov_mwf+nov_thu]:
#     process_date(filename)
process_date(('12','01'), make_y_df=True, has_header=True)